Chapter 2.a.iv.  Operation reductions
=======

**Operation** is the base class for most of the **Expression** classes defined in the **Prove-It** theory systems.  An **Operation** represents *operator(s)* acting upon *operand(s)*.  $1 + 5 - 1$, $1 < k \leq l$, and $x_1 \cdot \ldots \cdot x_n$ are a few examples of expressions that can be represented by classes derived from **Operation**.

There are essentially two kinds of reduction rules specific to **Operation** expressions that are applied in the process of performing a lambda application (beta reduction).

1. *Operation replacement*: The operation itself may be replaced, rather than just its operator(s) and/or operand(s), in two different ways.
    1. *Explicit operation replacement* is performed by replacing an operator with a **Lambda** expression.  In that case, the operation is replaced by the result of applying thislambda mapping to the operands (an internal beta reduction).  For example, $P(x, y)$ with $P:[(a, b) \mapsto a+b]$ becomes $x + y$.
    2. *Implicit operation replacement* is performed by replacing an operator with a designated **Literal** operator of an **Operation** class.  In that case, the operation is replaced by one of that derived class rather than the **Operation** class. For example, $P(x, y)$ with $P:+$ becomes $x + y$ with the correct `Add` type rather than a generic **Operation** type.
2. *Arity protection*: As an artifact of allowing either a single operand or multiple operands represented by an **ExprTuple**, rather than always having the operand(s) be an **ExprTuple**, we need to treat the following cases as special reduction rules.
    1. The **Operation** has a single operand that is replaced by an **ExprTuple**.  It must become an operation with a single **ExprTuple** operand rather than multiple operands.  For example, $f(x)$ with $x:(a, b, c)$ becomes $f((a, b, c))$ rather than $f(a, b, c)$.
    2. The **Operation** has an **ExprTuple** of multiple operands in an **ExprRange** that collapses to a single operand.  The outer **ExprTuple** must be eliminated consistent with haveing a single operand.  For example, $f(x_1, \ldots, x_n)$ with $n:1$ becomes $f(x_1)$ rather than $f((x_1))$.

In [ ]:
import proveit
%begin operation_reductions

## Creating operation expressions

Operation expressions come in various forms.  In any of its forms an **Operation** has two sub-expressions representing its *operator(s)* and *operand(s)* respectively.  Each of these may either be an **ExprTuple** or it represents a singular operator/operand.  There are two main types of formatting for an **Operation**, though any formatting rules may be defined by a class that derives from **Operation**.  These are *infix* and *function* formatting.

The default formatting of an **Operation** class is *infix* notation. For example,

In [ ]:
from proveit import Operation
from proveit._common_ import a, b, c
from proveit.number import Add

add_abc_opclass = Operation(Add._operator_, (a, b, c))

Note, however, that this is not the proper way to create an expression representing $a+b+c$ because it is not of type `Add`, we just borrowed the **Literal** operator for the `Add` operation called `Add._operator_`.

In [ ]:
add_abc_opclass.__class__

Instead, we should use the `Add` constructor.

In [ ]:
add_abc = Add(a, b, c)

In [ ]:
add_abc.__class__

There is also formatting available for multiple operators.  For example,

In [ ]:
from proveit.number import Less, LessEq, LesserSequence
seq = LesserSequence((Less._operator_, LessEq._operator_), (a, b, c))

We can see, below, that the `operators` and `operands` are both **ExprTuple** expression in this case.

In [ ]:
seq.exprInfo()

When you want *function* formatting, it is best to use the `Function` class that is derived from `Operation`:

In [ ]:
from proveit import Function
from proveit._common_ import f, g, x, y, z
Function(f, x)

In [ ]:
Function(f, (x, y, z))

It only makes sense to use *function* formatting when there is a single operator, so an error is raised if multiple operators are provided.

In [ ]:
try:
    Function((f, g), (x, y))
    assert False, "Expected ValueError."
except ValueError as e:
    print("Expected error:", e)    

## Operation replacement

We will demonstrate the two types of *operation replacement* in the following examples: explicit and implicit.

### Explicit operation replacement

To start our demonstration, let us define a **Lambda** expression which maps a variable $f$ to an expression in which $f$ is used as an operator.

In [ ]:
from proveit import Function, Lambda
from proveit.logic import Equals
from proveit._common_ import a, b, c, f, n, x, y, z
ab_substitution = Lambda(f, Equals(Function(f, a), Function(f, b)))

Now we just need to define a **Lambda** expression for replacing $f$.

In [ ]:
from proveit.number import Add, Mult
add_y_then_mult_z = Lambda(x, Mult(Add(x, y), z))

In [ ]:
ab_substitution.apply(add_y_then_mult_z)

Note that the $f(a)$ and $f(b)$ operations themselves were replaced according to the `add_y_then_mult_z` lambda mapping applied to $a$ and $b$ respectively.  Let's show an example with multiple operands.

In [ ]:
ab_commutation = Lambda(f, Equals(Function(f, [a, b]), Function(f, [b, a])))

In [ ]:
add_then_mult_z = Lambda((x, y), Mult(Add(x, y), z))

In [ ]:
ab_commutation.apply(add_then_mult_z)

### Implicit operation replacement

It is typical to derive classes from **Operation** that have a single corresponding *operator* that is a **Literal**.  This operator should be defined as the `_operator_` attribute of the derived class.  This indicates to **Prove-It** that this specific operator is tied to that specific class.  This knowlege is used in *implicit operation replacement*.  When an operator is replaced by `_operator_` of a particular **Operation** class, the **Operation** is reconstucted to by of that particular type.  For example.

In [ ]:
a_add_b_commutation = ab_commutation.apply(Add._operator_)

Is created using the proper `Add` class for constructing the left and right sides of the equation.

In [ ]:
a_add_b_commutation.lhs.__class__

In [ ]:
a_add_b_commutation.rhs.__class__

We can do the same using `Mult._operator_`:

In [ ]:
a_mult_b_commutation = ab_commutation.apply(Mult._operator_)

In [ ]:
a_mult_b_commutation.lhs.__class__

When verifying the correctness of a **Prove-It** proof (double-checking the derivation steps that we will discuss in future chapters), the specific type of **Operation** class of an expression is not important.  But this "reduction rule" is important in regards to the way **Prove-It** works as proofs are being generated.

## Arity protection

The arity of an operation, the number of its operands, must always be respected.  That is, in performing a lambda application and applying reduction rules, the number of operands must not change.  This requires special attention simply because a single operand, which is not an **ExprTuple** will not be wrapped in an **ExprTuple** as a convention for the expression DAGs to be more succinct (smaller).

### Protecting 1-arity

When we start with a function with arity 1 and replace its operand with an **ExprTuple**, we must not distinguish the result from a function with that **ExprTuple** as its operands.  We do this by making the new operands be an **ExprTuple** with a single element which is the **ExprTuple** replacing the original operand.  For example, starting with a simple **Lambda** expression ...

In [ ]:
from proveit._common_ import fx
operate_with_unary_f = Lambda(x, fx)

In [ ]:
operate_with_unary_f.body

... whose body is a function-formatted operation with a single operand that is the lambda parameters $x$ ...

In [ ]:
operate_with_unary_f.body.exprInfo()

... we can apply the map to an **ExprTuple** to obtain

In [ ]:
from proveit import ExprTuple
abc_on_unary = operate_with_unary_f.apply(ExprTuple(a, b, c))

which is distinguished from $f(a, b, c)$ by wrapping $(a, b, c)$ in an extra **ExprTuple**:

In [ ]:
abc_on_unary.exprInfo()

### Reducing to 1-arity

This situation can manifest itself in the reverse direction.  We can start with operand(s) of an indeterminant number via an **ExprRange** within an **ExprTuple** and then reduce it to a single operand which should no longer be wrapped in an **ExprTuple**.  For example, 

In [ ]:
from proveit.core_expr_types._common_ import x_1_to_n
nary_operation_map = Lambda(n, Function(f, x_1_to_n))

In [ ]:
nary_operation_map.body

In [ ]:
nary_operation_map.body.exprInfo()

We can `apply` our lambda expression to $1$ and allow the **ExprRange** to reduce, in a manner discussed in the previous chapter, to just a single $x_1$:

In [ ]:
from proveit.number import one
f_x1 = nary_operation_map.apply(one)

In addition to reducing the **ExprRange** to a single element, we must also eliminate the enclosing **ExprTuple** so that we have a single operand consistent with our convention.

In [ ]:
f_x1.exprInfo()

In [ ]:
%end operation_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>